In [1]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv('IMDB Dataset.csv')

In [4]:
df.sample(20)

,review,sentiment
10357,One of the most boring movies I've ever had to...,negative
6807,Valentine is a horrible movie. This is what I ...,negative
40867,While I certainly consider The Exorcist to be ...,negative
13520,Sam Mraovich should never be allowed to touch ...,negative
30684,Worst movie ever!! Its not clever or funny or ...,negative
35895,If Jacqueline McKenzie and John Lynch weren't ...,negative
24695,"Yes, it's flawed - especially if you're into H...",positive
46056,The lament and almost unbearable melancholy of...,positive
15401,So this is what actress Kim Basinger has succu...,negative
25762,"I thought it was going to be a lousy movie, ho...",positive


In [6]:
df.shape

(50000, 2)

In [7]:
df=df.iloc[:15000]

In [8]:
df.shape

(15000, 2)

In [10]:
df['sentiment'].value_counts()


sentiment
negative    7609
positive    7391
Name: count, dtype: int64

In [11]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [12]:
df.duplicated().sum()

39

In [13]:
df.drop_duplicates(inplace=True)

C:\Users\bagwe\AppData\Local\Temp\ipykernel_13988\3006716147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [15]:
df.shape

(14961, 2)

In [16]:
import re
def remove_html(text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', text)
    return cleaned_text
    

In [17]:
df['review']=df['review'].apply(remove_html)

C:\Users\bagwe\AppData\Local\Temp\ipykernel_13988\593116413.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(remove_html)


In [18]:
df['review'].head()

0    One of the other reviewers has mentioned that ...
1    A wonderful little production. The filming tec...
2    I thought this was a wonderful way to spend ti...
3    Basically there's a family where a little boy ...
4    Petter Mattei's "Love in the Time of Money" is...
Name: review, dtype: object

In [19]:
df['review']=df['review'].apply(lambda x:x.lower())

C:\Users\bagwe\AppData\Local\Temp\ipykernel_13988\2697927567.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review']=df['review'].apply(lambda x:x.lower())


In [20]:
df['review'].head()

0    one of the other reviewers has mentioned that ...
1    a wonderful little production. the filming tec...
2    i thought this was a wonderful way to spend ti...
3    basically there's a family where a little boy ...
4    petter mattei's "love in the time of money" is...
Name: review, dtype: object

In [21]:
from nltk.corpus import stopwords
sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

In [22]:
X=df.iloc[:,0:1]
y=df['sentiment']

In [23]:
X.shape,y.shape

((14961, 1), (14961,))

In [25]:
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()

In [26]:
y=le.fit_transform(y)

In [28]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [30]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33,random_state=42
                                              )

In [31]:
X_train.shape,X_test.shape

((10023, 1), (4938, 1))

In [32]:
from sklearn.feature_extraction.text import CountVectorizer


In [33]:
cv=CountVectorizer()

X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()


In [35]:
X_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [36]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

gnb.fit(X_train_bow,y_train)

GaussianNB()

In [37]:
y_pred = gnb.predict(X_test_bow)

from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_test,y_pred)

0.6462130417172944

In [38]:
confusion_matrix(y_test,y_pred)

array([[1835,  653],
       [1094, 1356]], dtype=int64)

In [39]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.8570271364925071

In [40]:
cv = CountVectorizer(ngram_range=(1,2),max_features=5000)

X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

rf = RandomForestClassifier()

rf.fit(X_train_bow,y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.8426488456865128

In [41]:
import pickle

In [42]:
pickle.dump(rf,open('model.pkl','wb'))